In [40]:
import sys
sys.path.insert(0,'/home/charles/@@@@@_jupyter/script')
# stdout= sys.stdout
# reload(sys)
# sys.setdefaultencoding('utf-8')
# sys.stdout = stdout
from MyEnv import Get_MyEnv
from NLP_QuestionType import *
from NLP_JiebaSegmentor import JiebaSegmentor, Get_JiebaSegmentor
from HelperDataApi import HelperDataApi
Get_MyEnv().env_cassandra
from envCassandraDAO import envCassandraDAO

In [41]:
import pandas as pd

In [52]:
DB_KEYSPACE = "nlp_keyspace"
dcard_article = "dcard_article"
dcard_response = "dcard_response"
dcard_article_relationship = "dcard_article_relationship"
dcard_response_relationship = "dcard_response_relationship"
word_article_mapping_relationship = "word_article_mapping_relationship"
dcard_article_food = "dcard_article_food"
dcard_response_food = "dcard_response_food"
word_article_mapping_food = "word_article_mapping_food"
board_keyword = "board_keyword"

In [43]:
dao = envCassandraDAO(Get_MyEnv().env_cassandra)
segmentor = Get_JiebaSegmentor()

In [27]:
# jieba_path = "/nfs/aq_test/jieba/"
# jieba_dict_path1 = jieba_path + "dict_taiwan.txt"
# jieba_dict_path2 = jieba_path +"userdict.txt"
# jieba_dict_path3 = jieba_path +"dict.txt.big"
# jieba_dict_path4 = jieba_path +"dict.txt.small"
# jieba_stopwords_path = jieba_path +"stopwords.txt"
# segmentor = JiebaSegmentor(jieba_dict_path1, 
#                            [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4], 
#                            stopwords = False, 
#                            stopwords_path=jieba_stopwords_path)

In [44]:
qt = QuestionTypeCheck(segmentor)

In [45]:
tp = qt.check_question_type("胖子")
tp.target, tp.label

(u'\u80d6\u5b50', 'what')

In [46]:
from gensim.corpora import *
from gensim.models import TfidfModel
from datetime import datetime
from gensim import models
tfidf_model_path = "/nfs/aq_dev/model/tfidf"
tfidf_model = TfidfModel.load(tfidf_model_path+"/tfidf.model")
dictionary = Dictionary.load(tfidf_model_path+"/tfidf_corpus_dict")

In [47]:
#過濾髒話
dirtywords_path = "/nfs/aq_dev/jieba/dcard_dirtywords.txt"
dirtyword_set = set()
with open(dirtywords_path,'r') as dirtywords:
    for dirtyword in dirtywords:
        dirtyword_set.add(dirtyword.strip('\n').decode('utf-8'))
        
dirty_word_rule = ''
for dw in dirtyword_set:
    dirty_word_rule = dirty_word_rule + dw + '|'
    
    
dirty_word_rule = '(' + dirty_word_rule[:len(dirty_word_rule) - 1 ] + ')'
# print dirty_word_rule

In [82]:
def find_keyword(text):
# #     test_cut_raw_0 = jieba.lcut(text)
#     test_cut_raw_0 =pseg.lcut(text)
#     text_subject_dict={}
#     for w,n in test_cut_raw_0:
#         print w + ' (' + n + ')'
#         text_subject_dict.update({w:n})
    
    if type(text) is not unicode:
        text = text.decode('utf-8')
        
    text = re.sub(dirty_word_rule, u"", text)
        
#     segmentor.stopwords=True

    text_subject_dict={}
    words_pd = segmentor.pseg_lcut(text)
    for index, row in words_pd.iterrows():
        text_subject_dict.update({row['word']:row['tag']})

    
    # jieba分词语料集 去除停用词
#     corpus0 = []
#     # load stopwords set
#     stopword_set = set()
#     with open(jieba_stopwords_path,'r') as stopwords:
#         for stopword in stopwords:
#             stopword_set.add(stopword.strip('\n').decode('utf8'))
#     for index, row in words_pd.iterrows():
#         if row['word'] not in stopword_set:
#             corpus0.append(row['word'])
    
    corpus0 = list(words_pd['word'])
    
    test_corpus_0 = dictionary.doc2bow(corpus0)
    test_corpus_tfidf_0 = tfidf_model[test_corpus_0]
    test_corpus_tfidf_0 = sorted(test_corpus_tfidf_0, key=lambda item: item[1], reverse=True)
    id2token = dict(zip(dictionary.token2id.values(), dictionary.token2id.keys()))
    result = []
    
    keyword_dict={}
    keyword_frames_list=[]
    word_list=[]
    tag_list=[]
    tfidf_list=[]
    for i in range(len(test_corpus_tfidf_0)):
        result.append({id2token[test_corpus_tfidf_0[i][0]]: test_corpus_tfidf_0[i][1]})
        print "--------------------------------------"
        word = id2token[test_corpus_tfidf_0[i][0]]
        tfidf = test_corpus_tfidf_0[i][1]
        print 'keyword: ' + word + '(' + text_subject_dict[word] + ') , tfidf: ' + str(tfidf)
        word_list.append(word)
        tag_list.append(text_subject_dict[word])
        tfidf_list.append(tfidf)
    
    keyword_pd=pd.DataFrame({"keyword":word_list,
                             "tag":tag_list,
                             "tfidf":tfidf_list})
    return keyword_pd
#         try:
#             w2v_predict(w2v_model, word)
#         except KeyError as er:
#             print 'no synonyms'
    

In [256]:
# TFIDF 字不會重複出現
#有名的豆漿
keyword_pd = find_keyword('有名的巧克力')
keyword_pd
word_list = list(keyword_pd["keyword"])
word_list

[u'\u5de7\u514b\u529b', u'\u6709\u540d']

In [257]:
cql = ("select * from " + board_keyword + ";")
board_keyword_df = dao.execCQLSelectToPandasDF(DB_KEYSPACE, cql)
board_keyword_df.sort_values(by=['board','count'], ascending=False).reset_index(drop=True)


,board,keyword,count
0,relationship,男友,246
1,relationship,喜歡,139
2,relationship,分手,112
3,relationship,女生,82
4,relationship,女友,78
5,relationship,男生,72
6,relationship,感情,65
7,relationship,朋友,47
8,relationship,知道,37
9,relationship,愛情,36


In [258]:
word_match_pd = board_keyword_df[board_keyword_df["keyword"].str.contains('|'.join(word_list))]
word_match_pd

,board,keyword,count
6,food,巧克力,55


In [259]:
keyword_pd = pd.merge(keyword_pd, word_match_pd, on="keyword")
keyword_pd

,keyword,tag,tfidf,board,count
0,巧克力,nr,0.755529,food,55


In [260]:
board_list = list(keyword_pd["board"].unique())
for bb in board_list:
    print bb

In [261]:
keyword_list = list(keyword_pd["keyword"])
for kk in keyword_list:
    print kk

In [262]:
# word_article_mapping

In [263]:
def filter_by_tag(tag):
    v_tag = ['v', 'vd', 'vg', 'vi', 'vn', 'vq', 'vt']
    n_tag = ['n', 'ng', 'nr', 'nrfg', 'nrt', 'ns', 'nt','nz']
    a_tag = ['a', 'ad', 'ag', 'an']
#         print tag['word'][0] + '(' + tag['tag'][0] + ')'
    if tag in v_tag or tag in n_tag or tag in a_tag:
        return True
    return False

def filter_by_tag_udf(row):
    v_tag = ['v', 'vd', 'vg', 'vi', 'vn', 'vq', 'vt']
    n_tag = ['n', 'ng', 'nr', 'nrfg', 'nrt', 'ns', 'nt','nz']
#     a_tag = ['a', 'ad', 'ag', 'an']
#         print tag['word'][0] + '(' + tag['tag'][0] + ')'
#     if row['tag'] in v_tag or row['tag'] in n_tag or row['tag'] in a_tag:
    if row['tag'] in v_tag or row['tag'] in n_tag:
        return True
    return False

In [264]:
"'" + "','".join(word_list) + "'"

u"'\u5de7\u514b\u529b','\u6709\u540d'"

In [265]:
# 查對應表
mapping_dict={}
mapping_frames_list=[]

# 只存名詞 and 動詞
keyword_pd['filter_by_tag'] = keyword_pd.apply(filter_by_tag_udf, axis=1)
keyword_pd = keyword_pd[keyword_pd['filter_by_tag'] == True]
print keyword_pd

# select * from nlp_keyspace.word_article_mapping_food where word in ('美食');
cql_where_in = board_list
cql = ("select * from nlp_keyspace.word_article_mapping_food where word in ('" +
       "','".join(word_list) + "');")
print cql
mapping_pd = dao.execCQLSelectToPandasDF(DB_KEYSPACE, cql)
top_n = 4
mapping_pd = mapping_pd.head(top_n)
mapping_pd

,word,article_id
0,巧克力,"[232816887, 233112679, 232989724, 233112679, 2..."
1,有名,[233353282]


In [266]:
# 統計哪個字出現最多次
article_list=[]
for index, row in mapping_pd.iterrows():
    #單個字去除重複文章id
    for item in list(set(row['article_id'])):
        article_list.append(item)
#         article_list.append(list(set(item)))
#         print type(item)
        
print str(len(article_list))
# article_list

In [267]:
from collections import Counter
result = Counter(article_list)
count_pd = pd.DataFrame(dict(result).items(), columns=['article_id', 'count'])
count_pd = count_pd.sort_values(by=['count'], ascending=False)
print len(count_pd)
count_pd

,article_id,count
0,233317831,1
40,233092515,1
29,232908371,1
30,232764952,1
31,232732756,1
32,232824122,1
33,232772492,1
34,232799221,1
35,232655612,1
36,232903216,1


# 至少要出現兩個字的文章==
top_pd = count_pd[count_pd['count'] > 1]
# top_pd =count_pd
# top_pd
count_list = sorted(list(set(top_pd['count'])), reverse=True)
# count_list
#
if len(count_list) == 0:
    print("AQ還在學習中......沒有相關問句")
    ppp
#     return jsonify(code=StateCode.Success,
#                    message="AQ還在學習中......沒有相關問句",
#                    data=[]
#                    )

# 至少要出現兩個字的文章查詢response,將其組合到answer_pd
response_frames_list = []
answer_pd = pd.DataFrame({})
for count in count_list:
#     logger.debug("count : {}".format(count))
    ans_pd = top_pd[top_pd['count'] == count]
    for index, row in ans_pd.iterrows():
        #         cql = ("select * from nlp_keyspace.dcard_response where article_id=" +
        #                str(row['article_id']) + ";")

        cql = ("select * from nlp_keyspace.dcard_response_food where article_id='" +
               str(row['article_id']) + "';")
        print(cql)
        pd_df = dao.execCQLSelectToPandasDF(DB_KEYSPACE, cql)
        response_frames_list.append(pd_df)

    if len(response_frames_list) > 0:
        answer_pd = pd.concat(response_frames_list).reset_index(drop=True)
        # 只選出前N個可能的回答
        if len(answer_pd) > 10:
            break

In [271]:
def has_dirty_word_udf(row):

    pattern = re.compile(dirty_word_rule)
    match = pattern.search(row['response'])
    if match:
        return True
    return False

# 過濾有髒話的答案
answer_pd['has_dirty_word'] = answer_pd.apply(has_dirty_word_udf, axis=1)
# answer_pd = answer_pd[answer_pd['has_dirty_word'] == False]

# if len(answer_pd) > 0:
#     # 依照讚數排序
#     answer_pd = answer_pd.sort_values(by=['like_count', 'create_date'], ascending=False)
    
answer_pd

,article_id,floor,board,crawl_date,create_date,like_count,response,response_clean,has_dirty_word
0,232908371,1,food,2020-03-31 07:50:24.578,2020-01-17T13:23:20.577Z,8,看到這篇文 我也突然好想吃喔,看到這篇文 我也突然好想吃喔,False
1,232908371,2,food,2020-03-31 07:50:24.585,2020-01-17T14:26:35.860Z,0,萊爾富有賣,萊爾富有賣,False
2,232908371,4,food,2020-03-31 07:50:20.875,2020-01-17T15:21:52.099Z,1,全聯有！,全聯有！,False
3,232908371,6,food,2020-03-31 07:50:24.438,2020-01-17T16:42:34.184Z,0,星光三越有,星光三越有,False
4,232908371,7,food,2020-03-31 07:50:20.889,2020-01-17T18:22:13.672Z,0,寶雅,寶雅,False
5,232908371,8,food,2020-03-31 07:50:24.597,2020-01-18T00:53:49.551Z,0,時代的眼淚,時代的眼淚,False
6,232908371,9,food,2020-03-31 07:50:24.432,2020-01-18T15:57:28.363Z,2,我覺得白雷神也蠻好吃的欸,我覺得白雷神也蠻好吃的欸,False
7,232908371,10,food,2020-03-31 07:50:24.451,2020-01-19T15:52:49.238Z,0,7-11買一送一,7-11買一送一,False
8,232908371,11,food,2020-03-31 07:50:20.894,2020-01-20T11:48:11.614Z,0,昨天去屈臣氏有看到,昨天去屈臣氏有看到,False
9,232764952,1,food,2020-03-31 10:07:17.623,2019-12-26T10:06:27.980Z,5,麴ㄑㄩˊ,麴ㄑㄩˊ,False


In [270]:
for i,r in answer_pd.reset_index().iterrows():
    print str(i) + '. ' + r['response']